In [1]:
# This program draws;
# a. the relation between
#    (X) the acquision date,
#    (Y) the selling date, and
#    (Z) the annualized return during the period
# b. the relation between
#    (X) the starting time of dollar cost avaraging strategy,
#    (Y) the ending time of the strategy, and
#    (Z) the annualized return during the period

In [2]:
import yfinance as yf

from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [3]:
# interval '1d' gets older data than '1mo'
df_SP500 = yf.download(['^GSPC'],interval='1d',period='max')

# calculate monthly
df_SP500['year'] = df_SP500.index.year
df_SP500['month'] = df_SP500.index.month

df_SP500.sort_index(inplace=True)
df_SP500.drop_duplicates(['year','month'],inplace=True)

df_SP500['month_passed'] = range(len(df_SP500))

df_SP500.reset_index(inplace=True)
df_SP500['Date'] = df_SP500['Date'].dt.strftime('%Y%m')

[*********************100%***********************]  1 of 1 completed


In [4]:
df_st = df_SP500.rename(columns={col:'st_'+col for col in df_SP500.columns})
df_en = df_SP500.rename(columns={col:'en_'+col for col in df_SP500.columns})

df_cross = df_st.join(df_en,how='cross').query('st_month_passed<en_month_passed')

df_cross['return'] = df_cross['en_Close'] / df_cross['st_Close']
df_cross['month_gap'] = df_cross['en_month_passed'] - df_cross['st_month_passed']
df_cross['return_annualized_a'] = (df_cross['return']) ** (12/df_cross['month_gap']) - 1

In [5]:
constant = 10000
df_SP500['quantity'] = constant / df_SP500['Close']

def b(st_month_passed:int,en_month_passed:int) -> float:
    
    df = df_SP500.query('@st_month_passed <= month_passed <= @en_month_passed').copy()
    df.at[df.index[-1],'quantity'] = 0

    latest_close = df.at[df.index[-1],'Close']

    return_total = latest_close * sum(df['quantity']) / (constant*(len(df)-1))

    return return_total ** (12/(len(df)-1)) - 1

l = []
for st,en in tqdm(df_cross[['st_month_passed','en_month_passed']].values):
    l.append(b(st,en))
df_cross['return_annualized_b'] = l

100%|██████████| 389403/389403 [11:27<00:00, 566.65it/s]


In [6]:
df_cross['return_annualized_a'] *= 100
df_cross['return_annualized_b'] *= 100

In [15]:
fig = plt.figure(figsize=(20,15))
plt.rcParams['font.size'] = 20
plt.style.use('dark_background')
sns.heatmap(
    df_cross.pivot(index='st_Date',columns='en_Date',values='return_annualized_a'),
    cmap=sns.color_palette("Spectral", as_cmap=True),
    square=True,
    vmax=15, vmin=-15, center=0
)
plt.title(f"Buying/Selling Month and Annualized Return [%] of S&P 500")
plt.xlabel("Buying Month")
plt.ylabel(f"Selling Month")

ticks = [i for i,(y,m) in enumerate(df_SP500[['year','month']].values) if y%2==0 and m==1]
labels = [y for i,(y,m) in enumerate(df_SP500[['year','month']].values) if y%2==0 and m==1]
plt.xticks(ticks=ticks, labels=labels)
plt.yticks(ticks=ticks, labels=labels)

for i,(y,m) in enumerate(df_SP500[['year','month']].values):
    if y%10==0 and m==1:
        plt.axvline(x=i,color="grey",linewidth=1)
        plt.axhline(y=i,color="grey",linewidth=1)

fig.savefig("../output/20230706_Start_End_and_Return_a.png")
plt.close()

In [14]:
fig = plt.figure(figsize=(20,15))
plt.rcParams['font.size'] = 20
plt.style.use('dark_background')
sns.heatmap(
    df_cross.pivot(index='st_Date',columns='en_Date',values='return_annualized_b'),
    cmap=sns.color_palette("Spectral", as_cmap=True),
    square=True,
    vmax=15, vmin=-15, center=0
)
plt.title(f"Dollar Cost Strategy Starting/Ending Month and Annualized Return [%] of S&P 500")
plt.xlabel("Starting Month")
plt.ylabel(f"Ending Month")

ticks = [i for i,(y,m) in enumerate(df_SP500[['year','month']].values) if y%2==0 and m==1]
labels = [y for i,(y,m) in enumerate(df_SP500[['year','month']].values) if y%2==0 and m==1]
plt.xticks(ticks=ticks, labels=labels)
plt.yticks(ticks=ticks, labels=labels)

for i,(y,m) in enumerate(df_SP500[['year','month']].values):
    if y%10==0 and m==1:
        plt.axvline(x=i,color="grey",linewidth=1)
        plt.axhline(y=i,color="grey",linewidth=1)

fig.savefig("../output/20230706_Start_End_and_Return_b.png")
plt.close()